In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import serial
import sys
import os
from serial.serialutil import SerialException
import glob
import asyncio
import serial_asyncio
import re
import time
import contextvars

In [ ]:
device = '/dev/tty.wchusbserial1420'

assistance = input('Do you need help connecting the device? y, N')
if assistance != 'y':
    ser = serial.Serial(device, 9600)
else:
    release_usb =input('Please disconnect the device now. Press any key to continue.')
    pre_devices = glob.glob('/dev/*')
    release_usb =input('Please reconnect the device now. Press any key to continue.')
    post_devices = glob.glob('/dev/*')

    new_devices = set(post_devices) - set(pre_devices)
    
    for device in new_devices:
        try:
            ser = serial.Serial(device, 9600)
        except SerialException:
            print(f'Not device {device}')

    ser.close()
    print(f'Contact with {ser.port}')

In [ ]:
%matplotlib ipympl

max_value_var = contextvars.ContextVar('max_value_var')
tmp_max = contextvars.ContextVar('tmp_max')
max_value_var.set(0)

class IOHelper(asyncio.Protocol):
    bounds = 200
    ymin, ymax = 65, bounds
    
    def connection_made(self, transport):
        self.transport = transport
        
        self.frame = np.zeros(self.bounds)
        transport.serial.rts = False  # You can manipulate Serial object via transport
        transport.write(b'Anything')  # Write serial data via transport
        
    def data_received(self, data):
        try:
            parsed = re.findall(r'([0-9]{2,3})', data.decode('utf-8'))
            ints = [int(x) for x in parsed]
            array = np.array(ints)
            data = np.median(array)
        except UnicodeDecodeError:
            data = np.median(self.frame[::-1][:5])
        self.frame[:-1] = self.frame[1:]
        self.frame[-1] = data
        
        self._draw_call(self.frame)
        tmp_max.set(self.frame.max())

    def connection_lost(self, exc):
        print('port closed')
        self.transport.loop.stop()

    def pause_writing(self):
        print('pause writing')
        print(self.transport.get_write_buffer_size())

    def resume_writing(self):
        print(self.transport.get_write_buffer_size())
        print('resume writing')
        
    def _canvas_setup(self):
        plt.figure() 
        ln, = plt.plot([])
        plt.ion()
        plt.show()
        #self.fig, self.plt, self.ax = fig, plt, ax
        self.plt, self.ln = plt, ln
        
    def _draw_call(self, data, last_max=None):
        max_value = max_value_var.get()
        plt, ln = figure_data.get()
        plt.plot()
        plt.axis([200,0, 50, 800])
        plt.title(f'Last max: {max_value}')
        plt.pause(0.1)
        ln.set_ydata(data[::-1])
        ln.set_xdata(range(len(data)))
        plt.draw()

In [ ]:
%matplotlib ipympl
plt.figure() 
ln, = plt.plot([])
plt.ion()
plt.show()

figure_data = contextvars.ContextVar('figure_data')
figure_data.set([plt, ln])

loop = asyncio.get_event_loop()
coro = serial_asyncio.create_serial_connection(loop, IOHelper, '/dev/tty.wchusbserial1420', baudrate=9600)

loop.run_until_complete(coro)
loop.run_forever()